In [1]:
# others
import pandas as pd
import numpy as np
import argparse
import datetime
from copy import deepcopy # Add Deepcopy for args
import pickle 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)


1.7.1
Populating the interactive namespace from numpy and matplotlib


In [2]:
# read file
train_x_df = pd.read_csv("./data/train_x_df.csv")
train_y_df = pd.read_csv("./data/train_y_df.csv")



In [2]:
''' greedy feature handleing'''
# test_df = train_x_df[train_x_df['volume'] != 0]
# test_df['rest_asset'] = test_df['volume'] - test_df['tb_base_av']
# test_df['greedy'] = test_df['tb_base_av'] / test_df['volume']

# test_df2 = test_df[['time', 'coin_index', 'open', 'high', 'low', 'close', 'volume', 'trades', 'tb_base_av','rest_asset', 'greedy']]
# test_df2[['coin_index','trades', 'volume', 'tb_base_av','rest_asset', 'greedy']].head()
# test_df2[test_df2['greedy'] == 1][['coin_index','trades', 'volume', 'tb_base_av','rest_asset', 'greedy']].head()

' greedy feature handleing'

In [6]:
# numpy arrary conversion

def df2d_to_array3d(df_2d):
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())

    # 2위 팀에서 임의로 넣어둠
    # sample_index = df_2d.sample_id.value_counts().index
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d


In [76]:
print(
        f'''
        coin_9_x_df {coin_9_x_df.shape}
        coin_9_y_df {coin_9_y_df.shape}
        merged_df ({coin_9_x_df.shape[0] + coin_9_y_df.shape[0]}, {coin_9_x_df.shape[1] })
        '''
    )


        coin_9_x_df (1667040, 12)
        coin_9_y_df (144960, 12)
        merged_df (1812000, 12)
        


In [82]:
def coindata_preprocessor(train_x_df = train_x_df, train_y_df = train_y_df, coin_num = 9):

    # df에서 지정된 코인 데이터만 추출
    coin_num_x_df = train_x_df[train_x_df['coin_index'] ==  coin_num]
    coin_num_y_df = train_y_df[train_y_df['coin_index'] ==  coin_num]

    # y dataframe time value에 1380 씩 adding
    coin_num_y_df.time = coin_num_y_df.time.copy() + 1380

    # x,y df merge하고 sample_id와 time 순으로 sorting
    merged_df = pd.concat([coin_num_x_df, coin_num_y_df])
    merged_df = merged_df.sort_values(by = ['sample_id','time']).reset_index(drop=True)

    # sample_id series orderly indexing
    sample_id_series = merged_df.sample_id.value_counts().reset_index().rename(columns = {"index" : "sample_id"})
    reset_index_series = sample_id_series.iloc[:,:1].sort_values(by = ['sample_id']).reset_index(drop=True)
    new_sample_dict = reset_index_series.reset_index().set_index('sample_id').to_dict()

    # sample_id value initialization
    merged_df['sample_id'] = merged_df['sample_id'].map(new_sample_dict['index'])

    return merged_df

In [42]:
merged_df.to_hdf('./data/merged_data.h5',  key = 'merged_df')

In [85]:
file_path = './data/merged_data.h5'
file_key = 'merged_df'

merged_coin_9_df = pd.read_hdf(file_path, file_key)

In [88]:
merged_coin_9_df

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,9,0.983614,0.983614,0.983128,0.983246,0.001334,10.650987,0.009855,0.000848,6.771755
1,0,1,9,0.983245,0.983612,0.982453,0.982693,0.001425,11.375689,0.016137,0.000697,5.565188
2,0,2,9,0.982694,0.983612,0.982403,0.983002,0.001542,12.301942,0.014166,0.000905,7.225459
3,0,3,9,0.983009,0.984848,0.983009,0.984486,0.002520,20.134695,0.021557,0.001171,9.353000
4,0,4,9,0.984233,0.984606,0.983612,0.984164,0.002818,22.515448,0.021434,0.001799,14.372534
...,...,...,...,...,...,...,...,...,...,...,...,...
1811995,1207,1495,9,1.005850,1.008398,1.005714,1.008084,0.018440,132.533844,0.166922,0.011676,83.914566
1811996,1207,1496,9,1.008109,1.008259,1.006929,1.007139,0.016465,118.390549,0.097126,0.006409,46.085777
1811997,1207,1497,9,1.007186,1.007418,1.005802,1.006216,0.006134,44.051334,0.060266,0.001914,13.744169
1811998,1207,1498,9,1.006092,1.007211,1.004988,1.005507,0.010358,74.361237,0.086474,0.004091,29.376879


In [122]:
class DatasetGenerater(Dataset):
    '''설명 : '''
    
    def __init__(self, dataframe, x_frames, y_frames):

        '''
        : param dataframe:   Raw data to load for making dataset
        : param x_frames:    the number of input day size
        : param y_frames:    the number of output day size
        '''
        self.dataframe = dataframe
        self.x_frames = x_frames
        self.y_frames = y_frames

        check_null = self.dataframe.isna().sum().sum()
        print("null clear") if check_null == 0 else print("null exist")
        
        
    def __len__(self):
        return len(self.dataframe) - (self.x_frames + self.y_frames) + 1
    
    
    def __getitem__(self, idx):
        idx += self.x_frames
        
        dataframe = self.dataframe.iloc[idx-self.x_frames:idx+self.y_frames]
        dataframe = dataframe.values
        
        X = dataframe[:self.x_frames]
        y = dataframe[self.x_frames:]
        
        return X, y